In [1]:
#!/usr/bin/env python
# coding: utf-8
# In[1]:
#!/usr/bin/env python
# coding: utf-8
# get_ipython().run_line_magic('matplotlib', 'inline')
import pandas as pd
import time
import numpy as np
from datetime import datetime
from sklearn.externals import joblib 
import os
import glob
from konlpy.tag import Mecab
import lightgbm as lgb
print(lgb.__version__)
from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.externals import joblib 
from sklearn.model_selection import StratifiedKFold, KFold
import gc
from tqdm import tqdm_notebook, tqdm
import json
from typing import NamedTuple
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

import warnings
warnings.filterwarnings(action='ignore')
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR
print(torch.__version__)
# from tools import eval_summary, save_feature_importance, merge_preds
from tools import EarlyStopping

device = torch.device('cpu')
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

/home/aiden/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


2.3.2
1.3.1
GeForce RTX 2070 SUPER


device(type='cuda', index=0)

In [2]:
# In[2]:
torch.set_num_threads(8)
torch.get_num_threads()

8

In [3]:
class DNNModel(torch.nn.Module):
    def __init__(self, input_size, dropout_probability=0.3, dims=32):
        super(DNNModel,self).__init__()
        act = torch.nn.ELU()
        dropout = torch.nn.Dropout(p=dropout_probability)

        self.m1 = torch.nn.Sequential(
            torch.nn.Linear(input_size, dims), torch.nn.BatchNorm1d(dims), act, dropout, 
            torch.nn.Linear(dims, dims), torch.nn.BatchNorm1d(dims), act, dropout, 
            torch.nn.Linear(dims, dims), torch.nn.BatchNorm1d(dims), act, dropout,
#             torch.nn.Linear(dims, dims), torch.nn.BatchNorm1d(dims), act, dropout,
#             torch.nn.Linear(dims, dims), torch.nn.BatchNorm1d(dims), act, dropout,
            torch.nn.Linear(dims, dims), torch.nn.BatchNorm1d(dims), act, dropout,
            torch.nn.Linear(dims, 1)
        )
    def forward(self, x):
        return self.m1(x)
 

In [46]:
df_score = joblib.load('df_pred_score_10.pkl')

In [39]:
tags = set()
for m in sorted(glob.glob('model/2020020*T*_e*')):
    print(m)
    m = m.replace('model/', '')
    m = m.split('_', 3)[0] + '_' + m.split('_', 3)[1] + '_' + m.split('_', 3)[2]
    tags.add(m + '_')


model/20200201T002710_e1_0.3232092261314392.model
model/20200201T002710_e1_0.3238092362880707.model
model/20200201T002710_e1_0.32838305830955505.model
model/20200201T002710_e1_0.5577053427696228.model
model/20200201T002710_e2_0.517207682132721.model
model/20200201T002710_e2_0.5455421805381775.model
model/20200201T002710_e2_0.9055352210998535.model
model/20200201T002710_e3_0.40459367632865906.model
model/20200201T002710_e3_0.4081472158432007.model
model/20200201T002710_e3_0.4227796792984009.model
model/20200201T002710_e3_0.5072051882743835.model
model/20200201T002710_e3_1.2683491706848145.model
model/20200201T002710_e4_0.38821637630462646.model
model/20200201T002710_e4_0.3883020877838135.model
model/20200201T002710_e4_0.39375483989715576.model
model/20200201T002710_e4_0.5474798083305359.model
model/20200201T002710_e4_1.024428129196167.model
model/20200201T020720_e1_0.3940942883491516.model
model/20200201T020720_e1_0.3948409855365753.model
model/20200201T020720_e1_0.39596104621887207.mod

In [43]:
tags = [
'20200201T002710_e',
'20200201T150530_e64_',
'20200201T163643_e96_',
'20200201T175741_e128_',
'20200201T221255_e192_',
'20200202T000925_e256_',
'20200202T010436_e288_',
'20200202T040846_e384_',
'20200202T050940_e416_',
]


In [27]:
tags = sorted(list(tags))
tags

['20200202T121309_e2_32_',
 '20200202T130017_e2_64_',
 '20200202T143051_e2_1024_',
 '20200202T150738_e2_96_',
 '20200202T154250_e2_128_',
 '20200202T163655_e2_512_',
 '20200202T170737_e2_192_',
 '20200202T174936_e2_384_',
 '20200202T182533_e2_256_',
 '20200202T190416_e2_416_']

In [48]:
for k, tag in enumerate(tags):
    
    for i in range(1, 5):
        layer_cols = [f'layer_{i}']
        fea_cols = [c for c in df_score.columns if c not in layer_cols and c[-1] == str(i)]
        
#         if k < 1:
#             fea_cols = fea_cols[:-1]
        print(fea_cols)
        print(layer_cols)
        model_path = sorted(glob.glob(f'model/{tag}{i}**'))[0]
        print(model_path)

        y_pred = joblib.load(model_path)(torch.Tensor(df_score[fea_cols].values).to(device))    
        df_score[layer_cols[0]] = y_pred.cpu().detach().numpy()
#         print(df_score[layer_cols + fea_cols])

    df_score.index.name = 'id'
    layer_cols = [c for c in df_score.columns if 'la' in c]
    df_score[layer_cols].to_csv(f'submit/{tag}_e.csv')
    print(f'submit/{tag}_e.csv')



['pred_0_1', 'pred_1_1', 'pred_2_1', 'pred_3_1', 'pred_4_1', 'pred_5_1', 'pred_6_1', 'pred_7_1', 'pred_8_1', 'pred_9_1']
['layer_1']
model/20200201T002710_e1_0.3232092261314392.model
['pred_0_2', 'pred_1_2', 'pred_2_2', 'pred_3_2', 'pred_4_2', 'pred_5_2', 'pred_6_2', 'pred_7_2', 'pred_8_2', 'pred_9_2']
['layer_2']
model/20200201T002710_e2_0.517207682132721.model
['pred_0_3', 'pred_1_3', 'pred_2_3', 'pred_3_3', 'pred_4_3', 'pred_5_3', 'pred_6_3', 'pred_7_3', 'pred_8_3', 'pred_9_3']
['layer_3']
model/20200201T002710_e3_0.40459367632865906.model
['pred_0_4', 'pred_1_4', 'pred_2_4', 'pred_3_4', 'pred_4_4', 'pred_5_4', 'pred_6_4', 'pred_7_4', 'pred_8_4', 'pred_9_4']
['layer_4']
model/20200201T002710_e4_0.38821637630462646.model
submit/20200201T002710_e_e.csv
['pred_0_1', 'pred_1_1', 'pred_2_1', 'pred_3_1', 'pred_4_1', 'pred_5_1', 'pred_6_1', 'pred_7_1', 'pred_8_1', 'pred_9_1']
['layer_1']
model/20200201T150530_e64_1_0.32047760486602783.model
['pred_0_2', 'pred_1_2', 'pred_2_2', 'pred_3_2', 

In [ ]:
# df_train = joblib.load('df_pred_train_10.pkl')
# df_valid = joblib.load('df_pred_valid_10.pkl')
df_score = joblib.load('df_pred_score_10.pkl')

In [ ]:
tags = set()
for m in sorted(glob.glob('model/2*T*_e*')):
    print(m)
    m = m.replace('model/', '')
    m = m.split('_', 2)[0] + '_' + m.split('_', 2)[1]
    if '20200201T133839' > m:
        tags.add(m[:-1])
    else:
        tags.add(m + '_')


In [ ]:
tags = sorted(list(tags))

tags

In [42]:

for k, tag in enumerate(tags):
    
    for i in range(1, 5):
        layer_cols = [f'layer_{i}']
        fea_cols = [c for c in df_score.columns if c not in layer_cols and c[-1] == str(i)]
        
        if k < 1:
            fea_cols = fea_cols[:-1]
        print(fea_cols)
        print(layer_cols)
        model_path = sorted(glob.glob(f'model/{tag}{i}**'))[0]
        print(model_path)

        y_pred = joblib.load(model_path)(torch.Tensor(df_score[fea_cols].values).to(device))    
        df_score[layer_cols[0]] = y_pred.cpu().detach().numpy()
#         print(df_score[layer_cols + fea_cols])

    df_score.index.name = 'id'
    layer_cols = [c for c in df_score.columns if 'la' in c]
    df_score[layer_cols].to_csv(f'submit/{tag}_e.csv')
    print(f'submit/{tag}_e.csv')



['layer_1_k1', 'layer_2_k1', 'layer_3_k1', 'layer_4_k1', 'layer_1_k11', 'layer_2_k11', 'layer_3_k11']
['layer_1']


IndexError: list index out of range

In [33]:
submit_csv = [
    'submit/e4.csv',
    'submit/20200202T040846_e384__e.csv',
    'submit/20200202T030727_e352__e.csv',
    'submit/20200202T021052_e320__e.csv',
    'submit/20200202T010436_e288__e.csv',
    'submit/20200202T000925_e256__e.csv',
    'submit/20200201T231055_e224__e.csv',
    'submit/20200201T221255_e192__e.csv',
    'submit/20200201T192753_e160__e.csv',
    'submit/20200201T175741_e128__e.csv',
    'submit/20200201T163643_e96__e.csv',
    'submit/20200201T150530_e64__e.csv',
    'submit/20200201T133839_e32__e.csv',
    'submit/20200201T113101_e_e.csv',
    'submit/20200201T093629_e_e.csv',
    'submit/20200201T020720_e_e.csv',
    'submit/20200201T002710_e_e.csv',
    'submit/20200131T225354_e_e.csv',
    'submit/20200202T182533_e2_256__e.csv',
# 'submit/20200202T174936_e2_384__e.csv',
# 'submit/20200202T170737_e2_192__e.csv',
# 'submit/20200202T163655_e2_512__e.csv',
# 'submit/20200202T154250_e2_128__e.csv',
# 'submit/20200202T150738_e2_96__e.csv',
# 'submit/20200202T143051_e2_1024__e.csv',
# 'submit/20200202T130017_e2_64__e.csv',
# 'submit/20200202T121309_e2_32__e.csv',
]

In [49]:
submit_csv = [
'submit/20200202T050940_e416__e.csv',
'submit/20200202T040846_e384__e.csv',
'submit/20200202T010436_e288__e.csv',
'submit/20200202T000925_e256__e.csv',
'submit/20200201T221255_e192__e.csv',
'submit/20200201T175741_e128__e.csv',
'submit/20200201T163643_e96__e.csv',
'submit/20200201T150530_e64__e.csv',
'submit/20200201T002710_e_e.csv',
]

In [50]:
dfs = [pd.read_csv(csv, index_col=0) for csv in submit_csv]

In [51]:
df = pd.concat(dfs)

In [52]:
df.groupby('id').mean().to_csv('e_final_{}.csv'.format(len(submit_csv)))

In [ ]:
# e_models = [
#     'model/e_1_20200131T202847_0.322307288646698.model',
# #     'model/e_1_20200131T221837_0.33561456203460693.model',
#     'model/e_2_20200131T213251_0.5193350911140442.model',
#     'model/e_3_20200131T214620_0.38977283239364624.model',
#     'model/e_4_20200131T211402_0.37712812423706055.model',
# #     'model/e_4_20200131T220102_0.3780273497104645.model',
# #     'model/e_3_20200131T214620_0.38977283239364624.model',
# #     'model/e_4_20200131T211402_0.37712812423706055.model',
# #     'model/e_3_20200131T205640_0.3976164758205414.model',
# # #     'model/e_2_20200131T204154_0.5304651260375977.model',
# #     'model/e_1_20200131T202847_0.322307288646698.model',
# ]
# for i in range(1, 5):
#     layer_cols = [f'layer_{i}']
#     fea_cols = [c for c in df_score.columns if c not in layer_cols and c[-1] == str(i)]
#     print(fea_cols)
#     print(layer_cols)
#     y_pred = joblib.load(e_models[i-1])(torch.Tensor(df_score[fea_cols].values).to(device))    
#     df_score[layer_cols[0]] = y_pred.cpu().detach().numpy()
#     print(df_score[layer_cols + fea_cols])

In [ ]:
df_score.index.name = 'id'
layer_cols = [c for c in df_score.columns if 'la' in c]
ts = datetime.now().strftime('%Y%m%dT%H%M%S')
df_score[layer_cols].to_csv(f'submit/{ts}_e.csv')